In [217]:
import pandas as pd
import numpy as np
import os
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import math
import re
import time
import os, psutil
from datetime import datetime

pd.set_option('display.max_columns', None)


#import linking table
linking_table = pd.read_csv("/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/2.PREPARE MASTERFILE AND FACTOR FILE/linking table.csv")
linking_table = linking_table[['PERMNO','cusip']]
linking_table = linking_table.rename(columns={'PERMNO':'permno'})

#import financial variable file
df = pd.read_csv("/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/2.PREPARE MASTERFILE AND FACTOR FILE/factor_vars_final.csv")

df = df.reset_index(drop=True)

df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day

#merge linking table and financial vars
df = df.merge(linking_table, how='left', on=["permno"])

In [2]:
#import IVOL files

#IVOL FILE 30 days maturity
df_ivol_30 = pd.read_csv('/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/7. IVOL/dcutx5pf0twrgw8i.csv')

df_ivol_30['year'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').year) for x in df_ivol_30['date']]
df_ivol_30['month'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').month) for x in df_ivol_30['date']]
df_ivol_30['day'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').day) for x in df_ivol_30['date']]

df_ivol_30['key'] = df_ivol_30['ticker'].astype(str) + df_ivol_30['month'].astype(str) + df_ivol_30['year'].astype(str)

maximum = df_ivol_30.groupby(['key'])['day'].max().reset_index()

dictio = dict (zip(maximum['key'], maximum['day']))

df_ivol_30['max'] = df_ivol_30['key'].map(dictio)

df_ivol_30 = df_ivol_30[df_ivol_30['day'] == df_ivol_30['max']]

df_ivol_30 = df_ivol_30.drop(['key', 'max'], axis=1)

df_ivol_30 = df_ivol_30.reset_index(drop=True)


#IVOL FILE 60 days maturity
df_ivol_60 = pd.read_csv('/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/7. IVOL/zmpw5nqyfqbtzxs3.csv')

df_ivol_60['year'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').year) for x in df_ivol_60['date']]
df_ivol_60['month'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').month) for x in df_ivol_60['date']]
df_ivol_60['day'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').day) for x in df_ivol_60['date']]

df_ivol_60['key'] = df_ivol_60['ticker'].astype(str) + df_ivol_60['month'].astype(str) + df_ivol_60['year'].astype(str)

maximum = df_ivol_60.groupby(['key'])['day'].max().reset_index()

dictio = dict (zip(maximum['key'], maximum['day']))

df_ivol_60['max'] = df_ivol_60['key'].map(dictio)

df_ivol_60 = df_ivol_60[df_ivol_60['day'] == df_ivol_60['max']]

df_ivol_60 = df_ivol_60.drop(['key', 'max'], axis=1)

df_ivol_60 = df_ivol_60.reset_index(drop=True)


#IVOL FILE 91 days maturity
df_ivol_91 = pd.read_csv('/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/7. IVOL/ct7p4umd1sml7opi.csv')

df_ivol_91['year'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').year) for x in df_ivol_91['date']]
df_ivol_91['month'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').month) for x in df_ivol_91['date']]
df_ivol_91['day'] = [np.int64(datetime.strptime(x, '%d/%m/%Y').day) for x in df_ivol_91['date']]

df_ivol_91['key'] = df_ivol_91['ticker'].astype(str) + df_ivol_91['month'].astype(str) + df_ivol_91['year'].astype(str)

maximum = df_ivol_91.groupby(['key'])['day'].max().reset_index()

dictio = dict (zip(maximum['key'], maximum['day']))

df_ivol_91['max'] = df_ivol_91['key'].map(dictio)

df_ivol_91 = df_ivol_91[df_ivol_91['day'] == df_ivol_91['max']]

df_ivol_91 = df_ivol_91.drop(['key', 'max'], axis=1)

df_ivol_91 = df_ivol_91.reset_index(drop=True)

In [218]:
#merge ivol files and financial vars

# MERGE IVOL 30 DAYS
call = df_ivol_30[df_ivol_30['cp_flag'] == 'C']

call['IV_DELTA'] = call.groupby(['cusip'])['impl_volatility'].pct_change(1)

call['Data_lagged'] = call.groupby(['cusip'])['IV_DELTA'].shift(-1)

call['IV_D_C_30'] = call['Data_lagged']*call['impl_volatility']

call = call[['cusip','year','month','IV_D_C_30']]


put = df_ivol_30[df_ivol_30['cp_flag'] == 'P']

put['IV_DELTA'] = put.groupby(['cusip'])['impl_volatility'].pct_change(1)

put['Data_lagged'] = put.groupby(['cusip'])['IV_DELTA'].shift(-1)

put['IV_D_P_30'] = put['Data_lagged']*put['impl_volatility']

put = put[['cusip','year','month','IV_D_P_30']]


df = df.merge(call, how='inner', on=["cusip",'year','month'])

df = df.merge(put, how='inner', on=["cusip",'year','month'])


# MERGE IVOL 60 DAYS
call = df_ivol_60[df_ivol_60['cp_flag'] == 'C']

call['IV_DELTA'] = call.groupby(['cusip'])['impl_volatility'].pct_change(1)

call['Data_lagged'] = call.groupby(['cusip'])['IV_DELTA'].shift(-1)

call['IV_D_C_60'] = call['Data_lagged']*call['impl_volatility']

call = call[['cusip','year','month','IV_D_C_60']]


put = df_ivol_60[df_ivol_60['cp_flag'] == 'P']

put['IV_DELTA'] = put.groupby(['cusip'])['impl_volatility'].pct_change(1)

put['Data_lagged'] = put.groupby(['cusip'])['IV_DELTA'].shift(-1)

put['IV_D_P_60'] = put['Data_lagged']*put['impl_volatility']

put = put[['cusip','year','month','IV_D_P_60']]


df = df.merge(call, how='inner', on=["cusip",'year','month'])

df = df.merge(put, how='inner', on=["cusip",'year','month'])


# MERGE IVOL 91 DAYS
call = df_ivol_91[df_ivol_91['cp_flag'] == 'C']

call['IV_DELTA'] = call.groupby(['cusip'])['impl_volatility'].pct_change(1)

call['Data_lagged'] = call.groupby(['cusip'])['IV_DELTA'].shift(-1)

call['IV_D_C_91'] = call['Data_lagged']*call['impl_volatility']

call = call[['cusip','year','month','IV_D_C_91']]


put = df_ivol_91[df_ivol_91['cp_flag'] == 'P']

put['IV_DELTA'] = put.groupby(['cusip'])['impl_volatility'].pct_change(1)

put['Data_lagged'] = put.groupby(['cusip'])['IV_DELTA'].shift(-1)

put['IV_D_P_91'] = put['Data_lagged']*put['impl_volatility']

put = put[['cusip','year','month','IV_D_P_91']]


df = df.merge(call, how='inner', on=["cusip",'year','month'])

df = df.merge(put, how='inner', on=["cusip",'year','month'])

/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/3811172907.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call['IV_DELTA'] = call.groupby(['cusip'])['impl_volatility'].pct_change(1)
/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/3811172907.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call['Data_lagged'] = call.groupby(['cusip'])['IV_DELTA'].shift(-1)
/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/3811172907.py:10: SettingWithCopyWarning: 
A value

In [219]:
#import sentiment files

sent_1 = pd.read_pickle('/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/5. SENTIMENT ANALYSIS 10KQ/10kq_sentiment_new_yiyangh_0_18680.pkl')

sent_2 = pd.read_pickle('/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/5. SENTIMENT ANALYSIS 10KQ/10kq_sentiment_new_yiyangh_18680_37361.pkl')

df_sent = pd.concat([sent_1,sent_2],axis=0)

df_sent = df_sent.drop(['text','Unnamed: 0','Unnamed: 0.1'], 1)

df_sent = df_sent.reset_index(drop=True)

df_sent


/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/1166139584.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sent = df_sent.drop(['text','Unnamed: 0','Unnamed: 0.1'], 1)


,symbol,cik,form,access_number,filed_date,report_url,filing_url,Y_Q,sic,permno,date,negative,neutral,positive,Positive,Negative,Polarity,Subjectivity
0,UNP,100885,10-K,0000950109-96-001749,1996-03-26,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR1,4011,48725,1996-03-26,13.00,130.00,37.00,435.00,615.00,-0.17,0.06
1,UNP,100885,10-Q,0000100885-96-000006,1996-05-15,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR2,4011,48725,1996-05-15,5.00,37.00,6.00,144.00,249.00,-0.27,0.05
2,UNP,100885,10-Q,0000100885-96-000008,1996-08-09,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR3,4011,48725,1996-08-09,4.00,50.00,16.00,123.00,195.00,-0.23,0.04
3,UNP,100885,10-Q,0000100885-96-000020,1996-11-13,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1996.QTR4,4011,48725,1996-11-13,3.00,55.00,14.00,135.00,226.00,-0.25,0.05
4,UNP,100885,10-K,0000100885-97-000002,1997-03-28,https://www.sec.gov/Archives/edgar/data/100885...,https://www.sec.gov/Archives/edgar/data/100885...,1997.QTR1,4011,48725,1997-03-28,7.00,127.00,37.00,400.00,471.00,-0.08,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37357,POOL,945841,10-Q,0000945841-21-000141,2021-07-29,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR3,5090,82515,2021-07-29,4.00,41.00,16.00,174.00,192.00,-0.05,0.06
37358,POOL,945841,10-Q,0000945841-21-000178,2021-10-28,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR4,5090,82515,2021-10-28,3.00,47.00,18.00,185.00,241.00,-0.13,0.06
37359,APA,1841666,10-Q,0001673379-21-000012,2021-05-07,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR2,1311,39490,2021-05-07,8.00,69.00,9.00,124.00,442.00,-0.56,0.06
37360,APA,1841666,10-Q,0001871133-21-000007,2021-08-05,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR3,1311,39490,2021-08-05,15.00,72.00,8.00,126.00,508.00,-0.60,0.06


In [220]:
# create a list with unique permno year combination to filter out unnecessary data from the financial vars file

df_sent['year'] = pd.DatetimeIndex(df_sent['date']).year

df_sent['month'] = pd.DatetimeIndex(df_sent['date']).month


z = df_sent.groupby('permno')['year'].unique()
z =  pd.DataFrame(z)

z.index.name = 'newhead'
z.reset_index(inplace=True)

In [221]:
#when 1 or 2 missing values this means that the code worked but there is only 1 or 2 sentiments found in the text
#So i fill these with 0

df_sent.loc[df_sent.isnull().sum(axis=1) == 1] = df_sent.loc[df_sent.isnull().sum(axis=1) == 1].fillna(0)
df_sent.loc[df_sent.isnull().sum(axis=1) == 2] = df_sent.loc[df_sent.isnull().sum(axis=1) == 2].fillna(0)

In [222]:
#fill other missing values with cross-sectional median

df_sent['negative'] = df_sent['negative'].fillna(df_sent.groupby(['year','month'])['negative'].transform('median'))

df_sent['neutral'] = df_sent['neutral'].fillna(df_sent.groupby(['year','month'])['neutral'].transform('median'))

df_sent['positive'] = df_sent['positive'].fillna(df_sent.groupby(['year','month'])['positive'].transform('median'))

df_sent['Positive'] = df_sent['Positive'].fillna(df_sent.groupby(['year','month'])['Positive'].transform('median'))

df_sent['Negative'] = df_sent['Negative'].fillna(df_sent.groupby(['year','month'])['Negative'].transform('median'))

df_sent['Polarity'] = df_sent['Polarity'].fillna(df_sent.groupby(['year','month'])['Polarity'].transform('median'))

df_sent['Subjectivity'] = df_sent['Subjectivity'].fillna(df_sent.groupby(['year','month'])['Subjectivity'].transform('median'))

In [223]:
# delete unnecessary information from financial vars file

v = df.loc[df.permno==z['newhead'][0]][df.loc[df.permno==z['newhead'][0]].year.isin(z['year'][0])]

for i in range(1,len(z)):
    y = list(z['year'][i])
    
    v2 = df.loc[df.permno==z['newhead'][i]][df.loc[df.permno==z['newhead'][i]].year.isin(y)]
    
    v = pd.concat([v,v2], ignore_index = True)
    
df = v
del v

In [224]:
#drop columns that have more than 10% missing values

x = df.describe().transpose()

x['pct_miss'] = 1- (x['count']/len(df))

x = x.sort_values(by=['pct_miss'],ascending=False)

cols_to_delete = list(x[x['pct_miss']>0.10].index)

df = df.drop(cols_to_delete, 1)

/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/1142473181.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(cols_to_delete, 1)


In [225]:
#impute rest of the missing values by cross-sectional median

for i in df.iloc[:,4:49].columns:
    df[str(i)] = df[str(i)].fillna(df.groupby(['year','month'])[str(i)].transform('median'))

In [226]:
# merge financial vars and sentiment file
df_merged = pd.merge(df, df_sent, on=['year', 'month', 'permno'], how='left')

In [227]:
#forward fill all sentiment variables --> they are quarterly updated, so each month within a quarter has 3-4x the same value

df_merged['negative'] = df_merged.groupby(['cusip'])['negative'].fillna(method='ffill')
df_merged['neutral'] = df_merged.groupby(['cusip'])['neutral'].fillna(method='ffill')
df_merged['positive'] = df_merged.groupby(['cusip'])['positive'].fillna(method='ffill')
df_merged['Positive'] = df_merged.groupby(['cusip'])['Positive'].fillna(method='ffill')
df_merged['Negative'] = df_merged.groupby(['cusip'])['Negative'].fillna(method='ffill')
df_merged['Polarity'] = df_merged.groupby(['cusip'])['Polarity'].fillna(method='ffill')
df_merged['Subjectivity'] = df_merged.groupby(['cusip'])['Subjectivity'].fillna(method='ffill')

In [228]:
df_merged.columns

Index(['Unnamed: 0', 'permno', 'gvkey', 'date_x', 'exchcd', 'market_value',
       'lag1.market_value', 'ret', 'D', 'D_12_sum', 'size', 'value', 'prof',
       'valprof', 'F_score', 'debtiss', 'repurch', 'nissa', 'growth',
       'aturnover', 'gmargins', 'ep', 'cfp', 'noa', 'inv', 'invcap', 'igrowth',
       'sgrowth', 'lev', 'roaa', 'roea', 'sp', 'mom', 'indmom', 'valmom',
       'valmomprof', 'mom12', 'momrev', 'lrrev', 'valuem', 'nissm', 'strev',
       'ivol', 'beta', 'season', 'indrrev', 'price', 'age', 'shvol', 'year',
       'month', 'day', 'cusip', 'IV_D_C_30', 'IV_D_P_30', 'IV_D_C_60',
       'IV_D_P_60', 'IV_D_C_91', 'IV_D_P_91', 'symbol', 'cik', 'form',
       'access_number', 'filed_date', 'report_url', 'filing_url', 'Y_Q', 'sic',
       'date_y', 'negative', 'neutral', 'positive', 'Positive', 'Negative',
       'Polarity', 'Subjectivity'],
      dtype='object')

In [229]:
#only keep relevant columns

df_merged = df_merged[['permno', 'date_x', 'exchcd', 'market_value', 'lag1.market_value', 'ret',
       'D', 'D_12_sum', 'size', 'value', 'prof', 'valprof', 'F_score',
       'debtiss', 'repurch', 'nissa', 'growth', 'aturnover', 'gmargins', 'ep',
       'cfp', 'noa', 'inv', 'invcap', 'igrowth', 'sgrowth', 'lev', 'roaa',
       'roea', 'sp', 'mom', 'indmom', 'valmom', 'valmomprof', 'mom12',
       'momrev', 'lrrev', 'valuem', 'nissm', 'strev', 'ivol', 'beta', 'season',
       'indrrev', 'price', 'age', 'shvol', 'year', 'month', 'day', 'cusip',
       'IV_D_C_30', 'IV_D_P_30', 'IV_D_C_60', 'IV_D_P_60', 'IV_D_C_91',
       'IV_D_P_91', 'negative',
       'neutral', 'positive', 'Positive', 'Negative', 'Polarity',
       'Subjectivity']]

df_merged = df_merged.rename(columns={'date_x': 'date'})

In [230]:
#import document similarity files

df_sim = pd.read_csv("/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/6. DOCUMENT SIMILARITY/SIMILARITY RESULTS FINAL.csv")

In [231]:
df_sim['year'] = pd.DatetimeIndex(df_sim['date']).year
df_sim['month'] = pd.DatetimeIndex(df_sim['date']).month

df_sim = df_sim.drop(['Unnamed: 0','date'], 1)

/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/3759851101.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sim = df_sim.drop(['Unnamed: 0','date'], 1)


In [232]:
df_merged = pd.merge(df_merged, df_sim, on=['year', 'month', 'permno'], how='left')


In [233]:
df_merged['cosine'] = df_merged.groupby(['cusip'])['cosine'].fillna(method='ffill')
df_merged['jaccard'] = df_merged.groupby(['cusip'])['jaccard'].fillna(method='ffill')

df_merged['symbol'] = df_merged.groupby(['cusip'])['symbol'].fillna(method='ffill')
df_merged['symbol'] = df_merged.groupby(['cusip'])['symbol'].fillna(method='bfill')

df_merged['form'] = df_merged.groupby(['cusip'])['form'].fillna(method='ffill')
df_merged['form'] = df_merged.groupby(['cusip'])['form'].fillna(method='bfill')

df_merged['sic'] = df_merged.groupby(['cusip'])['sic'].fillna(method='ffill')
df_merged['sic'] = df_merged.groupby(['cusip'])['sic'].fillna(method='bfill')

In [234]:
df_merged = df_merged[['date', 'year', 'month', 'day', 'permno', 'cusip', 'symbol', 'form', 'sic', 'IV_D_C_30', 'IV_D_P_30', 'IV_D_C_60', 'IV_D_P_60', 'IV_D_C_91',
       'IV_D_P_91', 'exchcd', 'market_value', 'lag1.market_value', 'ret',
       'D', 'D_12_sum', 'size', 'value', 'prof', 'valprof', 'F_score',
       'debtiss', 'repurch', 'nissa', 'growth', 'aturnover', 'gmargins', 'ep',
       'cfp', 'noa', 'inv', 'invcap', 'igrowth', 'sgrowth', 'lev', 'roaa',
       'roea', 'sp', 'mom', 'indmom', 'valmom', 'valmomprof', 'mom12',
       'momrev', 'lrrev', 'valuem', 'nissm', 'strev', 'ivol', 'beta', 'season',
       'indrrev', 'price', 'age', 'shvol','negative', 'neutral', 'positive', 'Positive', 'Negative', 'Polarity',
       'Subjectivity', 'cosine', 'jaccard']]

In [235]:
#load MDA sentiment data

df_sent_mda_1 = pd.read_csv("/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/5. SENTIMENT ANALYSIS 10KQ/MDA/MDA SENTIMENT 10K.csv")

df_sent_mda_2 = pd.read_csv("/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/5. SENTIMENT ANALYSIS 10KQ/MDA/MDA SENTIMENT 10Q.csv")


df_sent_mda = pd.concat([df_sent_mda_1,df_sent_mda_2],axis=0)


In [236]:
#organize the df

df_sent_mda = df_sent_mda.drop(['Unnamed: 0','Management’s Discussion and Analysis of Financial Condition and Results of Operations'], 1)

df_sent_mda = df_sent_mda.rename(columns={'negative': 'negative_mda','positive': 'positive_mda',
                                         'neutral': 'neutral_mda','Positive': 'Positive_mda',
                                         'Negative': 'Negative_mda','Polarity': 'Polarity_mda','Subjectivity': 'Subjectivity_mda'})


/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/435879505.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sent_mda = df_sent_mda.drop(['Unnamed: 0','Management’s Discussion and Analysis of Financial Condition and Results of Operations'], 1)


In [237]:
#when 1 or 2 missing values this means that the code worked but there is only 1 or 2 sentiments found in the text
#So i fill these with 0

df_sent_mda.loc[df_sent_mda.isnull().sum(axis=1) == 1] = df_sent_mda.loc[df_sent_mda.isnull().sum(axis=1) == 1].fillna(0)
df_sent_mda.loc[df_sent_mda.isnull().sum(axis=1) == 2] = df_sent_mda.loc[df_sent_mda.isnull().sum(axis=1) == 2].fillna(0)

In [238]:
df_sent_mda['year'] = pd.DatetimeIndex(df_sent_mda['date']).year
df_sent_mda['month'] = pd.DatetimeIndex(df_sent_mda['date']).month

In [239]:
#fill other missing values with cross-sectional median

df_sent_mda['negative_mda'] = df_sent_mda['negative_mda'].fillna(df_sent_mda.groupby(['year','month'])['negative_mda'].transform('median'))

df_sent_mda['neutral_mda'] = df_sent_mda['neutral_mda'].fillna(df_sent_mda.groupby(['year','month'])['neutral_mda'].transform('median'))

df_sent_mda['positive_mda'] = df_sent_mda['positive_mda'].fillna(df_sent_mda.groupby(['year','month'])['positive_mda'].transform('median'))

df_sent_mda['Positive_mda'] = df_sent_mda['Positive_mda'].fillna(df_sent_mda.groupby(['year','month'])['Positive_mda'].transform('median'))

df_sent_mda['Negative_mda'] = df_sent_mda['Negative_mda'].fillna(df_sent_mda.groupby(['year','month'])['Negative_mda'].transform('median'))

df_sent_mda['Polarity_mda'] = df_sent_mda['Polarity_mda'].fillna(df_sent_mda.groupby(['year','month'])['Polarity_mda'].transform('median'))

df_sent_mda['Subjectivity_mda'] = df_sent_mda['Subjectivity_mda'].fillna(df_sent_mda.groupby(['year','month'])['Subjectivity_mda'].transform('median'))

In [240]:
df_merged = pd.merge(df_merged, df_sent_mda, on=['year', 'month', 'permno'], how='left')

In [241]:
df_merged['negative_mda'] = df_merged.groupby(['cusip'])['negative_mda'].fillna(method='ffill')
df_merged['neutral_mda'] = df_merged.groupby(['cusip'])['neutral_mda'].fillna(method='ffill')
df_merged['positive_mda'] = df_merged.groupby(['cusip'])['positive_mda'].fillna(method='ffill')
df_merged['Positive_mda'] = df_merged.groupby(['cusip'])['Positive_mda'].fillna(method='ffill')
df_merged['Negative_mda'] = df_merged.groupby(['cusip'])['Negative_mda'].fillna(method='ffill')
df_merged['Polarity_mda'] = df_merged.groupby(['cusip'])['Polarity_mda'].fillna(method='ffill')
df_merged['Subjectivity_mda'] = df_merged.groupby(['cusip'])['Subjectivity_mda'].fillna(method='ffill')

In [242]:
df_merged = df_merged[['date_x', 'year', 'month', 'day', 'permno', 'cusip', 'symbol_x', 'form',
       'sic_x', 'IV_D_C_30', 'IV_D_P_30', 'IV_D_C_60', 'IV_D_P_60',
       'IV_D_C_91', 'IV_D_P_91', 'exchcd', 'market_value', 'lag1.market_value',
       'ret', 'D', 'D_12_sum', 'size', 'value', 'prof', 'valprof', 'F_score',
       'debtiss', 'repurch', 'nissa', 'growth', 'aturnover', 'gmargins', 'ep',
       'cfp', 'noa', 'inv', 'invcap', 'igrowth', 'sgrowth', 'lev', 'roaa',
       'roea', 'sp', 'mom', 'indmom', 'valmom', 'valmomprof', 'mom12',
       'momrev', 'lrrev', 'valuem', 'nissm', 'strev', 'ivol', 'beta', 'season',
       'indrrev', 'price', 'age', 'shvol', 'negative', 'neutral', 'positive', 'Positive',
       'Negative', 'Polarity', 'Subjectivity', 'cosine', 'jaccard','negative_mda', 'neutral_mda', 'positive_mda',
       'Positive_mda', 'Negative_mda', 'Polarity_mda', 'Subjectivity_mda']]

In [243]:
#rename some columns

df_merged = df_merged.rename(columns={'date_x': 'date', 'symbol_x':'symbol','sic_x':'sic'})
df_merged = df_merged.dropna()

In [244]:
df_merged['sum'] = df_merged['negative'] + df_merged['neutral'] + df_merged['positive']

for i in df_merged.iloc[:,60:63].columns:
    df_merged[i+'_pct'] = df_merged[i]/df_merged['sum']
    

In [271]:
df_merged['sum'] = df_merged['negative_mda'] + df_merged['neutral_mda'] + df_merged['positive_mda']

for i in df_merged.iloc[:,69:72].columns:
    df_merged[i+'_pct'] = df_merged[i]/df_merged['sum']

df_merged = df_merged.drop('sum', 1)

/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/2126205217.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_merged = df_merged.drop('sum', 1)


In [272]:
pd.set_option('display.max_rows', 80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

summary_stats = df_merged.describe().transpose().round(decimals=2)

print(summary_stats.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &     count &        mean &         std &       min &        25\% &         50\% &         75\% &           max \\
\midrule
year              & 107079.00 &     2009.21 &        6.84 &   1996.00 &    2004.00 &     2010.00 &     2015.00 &       2021.00 \\
month             & 107079.00 &        6.52 &        3.44 &      1.00 &       4.00 &        7.00 &       10.00 &         12.00 \\
day               & 107079.00 &       30.44 &        0.80 &     28.00 &      30.00 &       31.00 &       31.00 &         31.00 \\
permno            & 107079.00 &    52265.03 &    27053.29 &  10078.00 &   24221.00 &    53065.00 &    77659.00 &      93422.00 \\
sic               & 107079.00 &     4538.50 &     1821.08 &    100.00 &    3089.00 &     4210.00 &     6020.00 &       9997.00 \\
IV\_D\_C\_30         & 107079.00 &        0.00 &        0.08 &     -2.57 &      -0.03 &       -0.00 &        0.03 &          2.52 \\
IV\_D\_P\_30         & 107079.00 &       -0.00 &      

/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/3049816162.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summary_stats.to_latex())


In [273]:
df_scaled = df_merged.copy()

cols_to_scale = list(df_merged.iloc[:,15:83].columns)


from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

df_scaled[cols_to_scale] = min_max_scaler.fit_transform(df_scaled[cols_to_scale])


In [276]:
summary_stats = df_scaled.describe().transpose().round(decimals=2)

print(summary_stats.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &     count &     mean &      std &      min &      25\% &      50\% &      75\% &      max \\
\midrule
year              & 107079.00 &  2009.21 &     6.84 &  1996.00 &  2004.00 &  2010.00 &  2015.00 &  2021.00 \\
month             & 107079.00 &     6.52 &     3.44 &     1.00 &     4.00 &     7.00 &    10.00 &    12.00 \\
day               & 107079.00 &    30.44 &     0.80 &    28.00 &    30.00 &    31.00 &    31.00 &    31.00 \\
permno            & 107079.00 & 52265.03 & 27053.29 & 10078.00 & 24221.00 & 53065.00 & 77659.00 & 93422.00 \\
sic               & 107079.00 &  4538.50 &  1821.08 &   100.00 &  3089.00 &  4210.00 &  6020.00 &  9997.00 \\
IV\_D\_C\_30         & 107079.00 &     0.00 &     0.08 &    -2.57 &    -0.03 &    -0.00 &     0.03 &     2.52 \\
IV\_D\_P\_30         & 107079.00 &    -0.00 &     0.08 &    -2.03 &    -0.03 &    -0.00 &     0.03 &     2.06 \\
IV\_D\_C\_60         & 107079.00 &     0.00 &     0.07 &    -2.57 &    -0.03 &   

/var/folders/qs/_5gz88hj5tn_l9mj01ndrgb40000gn/T/ipykernel_8942/2538146907.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(summary_stats.to_latex())


In [280]:
df_merged.to_csv('/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/FINAL DATASET/Dataset_unscaled.csv')
df_scaled.to_csv('/Users/thomasvanhoorn/Documents/THESIS NEW/DATA/FINAL DATASET/Dataset_scaled.csv')